In [360]:
import pandas as pd
import plotly.express as px

In [361]:
dataframes = {}
years = [2011, 2013, 2015, 2017, 2019, 2021]
for year in years:
    dataframes[year] = pd.read_csv(f"./data/food_security_{year}.csv")

In [362]:
merged = pd.concat(dataframes.values(), ignore_index=True)
merged

,State,Food Security,Children Food Security,Class of Worker,Race of Respondent,Family Type,Industry Type,Income,Year
0,AL,Food Secure High or Marginal Food Security,Food Secure High or Marginal Food Security,NaN,White only,Primary family,"In Universe, Met No Conditions To Assign",02: 5000 To 7499,2011
1,AL,Food Secure High or Marginal Food Security,Food Secure High or Marginal Food Security,NaN,White only,Primary family,"In Universe, Met No Conditions To Assign",02: 5000 To 7499,2011
2,AL,Food Secure High or Marginal Food Security,Food Secure High or Marginal Food Security,"Private, For Profit",White only,Primary family,Wholesale and retail trade,15: 100000 To 149999,2011
3,AL,Food Secure High or Marginal Food Security,Food Secure High or Marginal Food Security,"Private, For Profit",White only,Primary family,Educational and health services,15: 100000 To 149999,2011
4,AL,Food Secure High or Marginal Food Security,Food Secure High or Marginal Food Security,NaN,White only,Primary family,"In Universe, Met No Conditions To Assign",15: 100000 To 149999,2011
...,...,...,...,...,...,...,...,...,...
267071,CT,Food Secure High or Marginal Food Security,Food Secure High or Marginal Food Security,"Private, For Profit",White only,Primary family,Professional and business services,15: 100000 To 149999,2021
267072,CT,Food Secure High or Marginal Food Security,Food Secure High or Marginal Food Security,NaN,White only,Primary family,"In Universe, Met No Conditions To Assign",15: 100000 To 149999,2021
267073,OH,Food Secure High or Marginal Food Security,Food Secure High or Marginal Food Security,"Private, For Profit",Asian only,Primary family,Manufacturing,16: 150000 or More,2021
267074,OH,Food Secure High or Marginal Food Security,Food Secure High or Marginal Food Security,"Private, For Profit",White only,Primary family,Transportation and utilities,16: 150000 or More,2021


In [384]:
year_family_foodsec_grp = merged.groupby(['Year', 'Food Security'])['Food Security'].count().rename("yearly_family_food_security_rate").groupby(level = 0).transform(lambda x: round(100 * x/x.sum(), 2))
year_child_foodsec_grp = merged.groupby(['Year', 'Children Food Security'])['Children Food Security'].count().rename("yearly_child_food_security_rate").groupby(level = 0).transform(lambda x: round(100 * x/x.sum(), 2))

most_recent_df = merged[merged['Year'] == 2021]
state_child_foodsec_grp = most_recent_df.groupby(['State', 'Children Food Security'])['Children Food Security'].count().rename("state_child_food_security_rate").groupby(level = 0).transform(lambda x: round(100 * x/x.sum(), 2))
ind_child_foodsec_grp = most_recent_df.groupby(['Industry Type', 'Children Food Security'])['Children Food Security'].count().rename("ind_child_food_security_rate").groupby(level = 0).transform(lambda x: round(100 * x/x.sum(), 2))
race_foodsec_grp = merged.groupby(['Race of Respondent', 'Food Security'])['Food Security'].count().rename("race_food_security_rate").groupby(level = 0).transform(lambda x: round(100 * x/x.sum(), 2))
labels={
    'Family Type':'Family Type', 
    'yearly_family_food_security_rate':'Family Food Security (% of respondents)', 
    'yearly_child_food_security_rate':'Children Food Security (% of respondents)', 
    'state_child_food_security_rate' : 'Children Food Security (% of respondents)',
    'ind_child_food_security_rate' : 'Children Food Security (% of respondents)',
    'race_food_security_rate' : 'Family Food Security (% of respondents)'
    }


In [364]:
#most_recent_df.iloc[1]['Race of Respondent']
most_recent_df[most_recent_df['Race of Respondent'] == 'W-A-HP'].count()

State                     48
Food Security             48
Children Food Security    48
Class of Worker           11
Race of Respondent        48
Family Type               48
Industry Type             48
Income                    48
Year                      48
dtype: int64

In [385]:
fig = px.bar(year_family_foodsec_grp.reset_index(), 
             x='Year', 
             y='yearly_family_food_security_rate', 
             labels=labels, 
             color='Food Security', 
             barmode='group', 
             title='''Family Food Security in the United States has been improving YoY from 2011-2021.
                    ''',
             text_auto=True,             
             color_discrete_sequence=[ "#fcae91", "#fb6a4a", "#cb181d"])
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

In [386]:
df3 = race_foodsec_grp.unstack()
labels={'value':'Food Security (% of respondents)', 'variable':'Food Security'}
fig = px.bar(df3.reset_index().sort_values(by=['Low Food Security', 'Very Low Food Security'], ascending=False),
             x='Race of Respondent', 
             y=['Low Food Security', 'Very Low Food Security'],
             labels=labels, 
             barmode='stack', 
             title='''Food Security by Race.
                    ''',
             text_auto=True,             
             color_discrete_sequence=[ "#fcae91", "#fb6a4a", "#cb181d"])
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

In [387]:
fig = px.bar(year_child_foodsec_grp.reset_index(), 
             x='Year', 
             y='yearly_child_food_security_rate', 
             labels=labels, 
             color='Children Food Security', 
             barmode='group', 
             title='''Children Food Security in the United States has been improving YoY over the period 2011-2021.
                    <br>However, in 2021, there was a slight increase in Very Low Food Security.
                    ''',
             text_auto=True,      
             color_discrete_sequence=[ "#fcae91", "#fb6a4a", "#cb181d"])
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

In [390]:
df3 = state_child_foodsec_grp.unstack()
labels={'value':'Children Food Security (% of respondents)', 'variable':'Children Food Security'}
fig = px.bar(df3.reset_index().nlargest(10, 'Low Food Security'),
             x='State', 
             y=['Low Food Security', 'Very Low Food Security'],
             labels=labels, 
             title='''Bottom 10 States for Children Food Security, 2011-2021 (avg)''',
             barmode='stack', 
             orientation='v',
             text_auto=True,             
             color_discrete_sequence=[ "#fcae91", "#fb6a4a", "#cb181d"]
             )
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

In [392]:
df3 = state_child_foodsec_grp.unstack()
labels1={'value':'Children Food Security (% of respondents)', 'variable':'Children Food Security'}
fig = px.bar(df3.reset_index().nsmallest(10, 'Low Food Security'),
             x='State', 
             y=['Low Food Security', 'Very Low Food Security'],
             labels=labels1, 
             title='''Top 10 States for Children Food Security, 2011-2021 (avg)''',
             barmode='stack', 
             orientation='v',
             text_auto=True,       
             color_discrete_sequence=[ "#fcae91", "#fb6a4a", "#cb181d"]
             )
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

In [394]:
df3 = state_child_foodsec_grp.unstack()
labels1={'value':'Children Food Security (% of respondents)', 'variable':'Children Food Security'}
fig = px.bar(df3.reset_index().sort_values(by=['Low Food Security'], ascending=True),
             x='State', 
             y=['Low Food Security', 'Very Low Food Security'],
             labels=labels1, 
             title='''Children Food Security, 2011-2021 (avg)''',
             barmode='stack', 
             orientation='v',
             text_auto=True,
             color_discrete_sequence=[
                 "#fcae91", "#fb6a4a", "#cb181d"]
             )
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()